# PD3 - Jan Smoleń

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
import category_encoders as ce
import sklearn.metrics as metrics
import statistics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
df=pd.read_csv("australia.csv")

W celu przyśpieszeniea późniejszego tuningowania hiperparametrów ograniczę liczbę rekordów.

In [ ]:
df=df.head(5000)

In [ ]:
from sklearn.model_selection import train_test_split
X=df.drop(["RainTomorrow"], axis=1)
y=df["RainTomorrow"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state = 42)

## SVM
Pierwszym testowanym przez nas modelem będzie SVM.

In [ ]:
svm_base=SVC(random_state=42)
svm_base.fit(X_train, y_train)
preds=svm_base.predict(X_test)


In [ ]:
accuracy_score(preds,y_test)

Jak widzimy, SVM z domyślnymi hiperparametrami osiąga accuracy score powyżej 77%. Spróbujemy teraz znaleźć dobrą kombinacje hiperparametrów gamma i C korzystając z narzędzia GridSearchCV.

In [ ]:
svm_tuned=SVC(random_state=42)
c=[]  # wartości parametru C
gamma=[]  #wartości parametru gamma 
for i in range(-4, 5):      # orientacyjne wartości na podstawie informacji znalezionych w internecie
    c.append(10**i)
for i in range(-4, 5):
    gamma.append(10**i)
gamma.append("auto")
gamma.append("scale")
params = [{'C': c,   
        'gamma': gamma}]
gs_svm=GridSearchCV(svm_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_svm.fit(X_train, y_train)
gs_svm.best_params_


In [ ]:
svm_acc=accuracy_score(gs_svm.predict(X_test),y_test)
svm_acc

Tuning dwóch hiperparametrów pozwala zatem na poprawienie accuracy score modelu o prawie 10% przy wartościach C=100, gamma=0.0001. Gdyby celem zadania było znalezienie optymalnych parametrów to moglibyśmy poszukać także w okolicach tych wartości oraz zmodyfikować atrybut kernel.

## XGBoost

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 1613, use_label_encoder=False)
xgb_model.fit(X_train, y_train)


In [ ]:
accuracy_score(xgb_model.predict(X_test), y_test)

Surowy XGBoost daje znacznie lepszy wynik accuracy score niż SVM z domyślnymi parametrami.

In [ ]:
xgb_tuned=xgb.XGBClassifier(objective = "binary:logistic", seed = 1613, use_label_encoder=False)
eta=[]  #wartości parametru eta 
max_depth=[]
for i in range(10):
    eta.append(0.01+0.03) 
    max_depth.append(i)
params = [{'eta': eta,
        'max_depth': max_depth}]
gs_xgb=GridSearchCV(xgb_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_xgb.fit(X_train, y_train)
gs_xgb.best_params_

In [ ]:
xgb_acc=accuracy_score(gs_xgb.predict(X_test), y_test)
xgb_acc

W tym przypadku nie udało się polepszyć wyników modelu poprzez tuning hiperparametrów max_depth i eta.

## Random Forest

In [ ]:
rfc = RandomForestClassifier(random_state=16)
rfc.fit(X_train,y_train)

In [ ]:
accuracy_score(rfc.predict(X_test),y_test)

Czyli takie same accuracy jak używając XGBoosta.

In [ ]:
rfc_tuned=RandomForestClassifier(random_state=16)
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 1000, num = 5)] # przykładowe wartości znalezione w internecie 
max_depth = [int(x) for x in np.linspace(5, 55, num = 5)]
             
params = [{'n_estimators': n_estimators,
        'max_depth': max_depth}]
gs_rfc=GridSearchCV(rfc_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_rfc.fit(X_train, y_train)
gs_rfc.best_params_             

In [ ]:
rfc_acc=accuracy_score(gs_rfc.predict(X_test),y_test)
rfc_acc

Czyli udało się trochę polepszyć wynik naszego modelu. 

## Ocena jakości modeli 

### Accuracy Score

In [ ]:
scores=[]
labels=[]
scores.append(svm_acc)
labels.append("SVM")
scores.append(xgb_acc)
labels.append("XGB")
scores.append(rfc_acc)
labels.append("RFC")

In [ ]:
pd.DataFrame({"Accuracy Score": scores}, index=labels)

### Confusion Matrix
#### SVM

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, gs_svm.predict(X_test)).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

#### XGB

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, gs_xgb.predict(X_test)).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

#### RFC

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, gs_rfc.predict(X_test)).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

### ROC

In [ ]:
gs_svm
plt.figure(figsize=(12,10))
classifiers = [gs_svm, gs_xgb, gs_rfc]
labels=["SVM", "XGB", "RFC"]
ax = plt.gca()
for i in range(3):
    metrics.plot_roc_curve(classifiers[i], X_test, y_test, ax=ax, name=labels[i])

Biorąc pod uwagę powyższe oceny jakości klasyfikatorów, w tym konkretnym przypadku najlepszym z nich wydaje się **SVM** z wytuningowanymi hiperparametrami C i gamma. Być może inne wyniki byśmy otrzymali przeprowadzając wcześniej feature engineering.